In [ ]:

# sessionInfo()
suppressPackageStartupMessages({
    library(DESeq2)
    library(dplyr)
    library(ggplot2)
    library(ggrepel)
    library(ggpubr)
    library(RColorBrewer)
    library(cowplot)
    library(Matrix)
    library(stringr)
    library(DESeq2)
    library(edgeR)
    library(data.table)
    library(tximport)
    library(tidyr)
    library(readr)
    library(DESeq2)
    library(EnsDb.Hsapiens.v86)
    library(AnnotationHub)
    library(ggplot2)
    library(pheatmap)
    library(RColorBrewer)
    library(forcats)
    library(goseq)
    library(geneLenDataBase)
    library(org.Hs.eg.db)
    library(clusterProfiler)
    library(enrichplot)
    library(DOSE)
    library(TxDb.Hsapiens.UCSC.hg38.knownGene)
#         library(ggrastr)
    library(tictoc)
})

TxDb= TxDb.Hsapiens.UCSC.hg38.knownGene





### Input File Names

In [ ]:
indir = '/nfs/lab/projects/islet_multiomics_stress_CoCl2/pipeline/bulk_islet_rna/'
outdir = '/nfs/lab/projects/islet_multiomics_stress_CoCl2/pipeline/DESeq/rna/redo_bulkIslet_RNA_fixedAnno/'
outdir_bulk = outdir
ALPHA = 0.1

infilename = paste0(indir, 
                    'bulk_islet_rna.hg38.Donor_plus_Treatment.Replicates_MERGED_by_sum.ANNOTATED.FILT_2Plus_Reads_per_Treat.with_TPMs.countsfromTxImport.dds.rds')




In [ ]:
tic('read')
dds <- readRDS(infilename)
toc()

In [ ]:
as.data.frame(colData(dds))

In [ ]:
dds$treatment <- factor(dds$Treatment, levels = c("Unt", "CoCl2"))

# Run DESeq

In [ ]:
dds_postDESeq <- DESeq(dds, fitType="parametric")    

In [ ]:
tic('24h save')
ddsoutfilename = paste0(outdir, 'bulk_islet_rna.hg38.Donor_plus_Treatment.Replicates_MERGED_by_sum.ANNOTATED.FILT_2Plus_Reads_per_Treat.with_TPMs.countsfromTxImport.afterDESeq.fittype_parametric.rds')
saveRDS(dds_postDESeq, file = ddsoutfilename)
toc()


# Get Results

In [ ]:
rowdata = as.data.frame(rowData(dds_postDESeq))

In [ ]:
head(rowdata)

In [ ]:

tablename = paste0(outdir, 'bulk_islet_rna.hg38.Donor_plus_Treatment.Replicates_MERGED_by_sum.ANNOTATED.FILT_2Plus_Reads_per_Treat.with_TPMs.postDESeq.Gene_RowData.txt')
tic()
write.table(rowdata, tablename, quote=FALSE, sep='\t', col.names = TRUE, row.names=TRUE)
toc()


In [ ]:
res <- results(dds_postDESeq, contrast=c('Treatment', 'CoCl2', 'Unt'), alpha = ALPHA)
res <- as.data.frame(res)
# res <- cbind(res, rowdata)
# res <- as.data.frame(res)
# head(as.data.frame(res))
# dim(res)
# dim(rowdata)
# rownames(rowdata) == rownames(res)
res_rowdata = cbind(res, rowdata)
# dim(res_rowdata)

# length(rownames(res_rowdata))
# length(unique(rownames(res_rowdata)))



In [ ]:

tic('24h save')
resoutfilename = paste0(outdir_bulk, 'bulk_islet_rna.hg38.Donor_plus_Treatment.Replicates_MERGED_by_sum.ANNOTATED.FILT_2Plus_Reads_per_Treat.with_TPMs.postDESeq.RESULTS.withRowdata.txt')
write.table(res_rowdata, resoutfilename, quote=FALSE, sep='\t', col.names = TRUE, row.names=TRUE)
toc()



In [ ]:
res$gene_name = res_rowdata$gene_name
res$gene_id = res_rowdata$gene_id


In [ ]:
res_filt = res %>% dplyr::filter(  pvalue < 0.05 )








In [ ]:
res_filt = res_filt %>% dplyr::arrange(padj)

In [ ]:
dim(res_filt)

In [ ]:
head(res_filt)

In [ ]:

tic('24h save')
resoutfilename = paste0(outdir_bulk, 'bulk_islet_rna.hg38.Donor_plus_Treatment.Replicates_MERGED_by_sum.ANNOTATED.FILT_2Plus_Reads_per_Treat.with_TPMs.postDESeq.RESULTS.pval_0.05.txt')
write.table(res_filt, resoutfilename, quote=FALSE, sep='\t', col.names = TRUE, row.names=TRUE)
toc()



In [ ]:

tablename = paste0(indir, 'endoC.hg38.', testName, '.TPMs.unfiltered.txt')
tic()
tpm_final_72 = read.table(tablename)
toc()




tpm_final_72_sort = tpm_final_72[rownames(res_rowdata),]
tpm_final_72_sort$gene_width_from_gencode41.gene_name = NULL
tpm_final_72_sort$gene_width_from_gencode41.row_name = NULL
tpm_final_72_sort$gene_width_from_gencode41.gene_width_from_gencode41 = NULL
tpm_final_72_sort$gene_id = NULL
tpm_final_72_sort$gene_gene_nameid = NULL
tpm_final_72_sort$gene_width_from_gencode41.gene_id = NULL
tpm_final_72_sort$gene_name = NULL
head(tpm_final_72_sort)

In [ ]:

tablename = paste0(indir, 'bulk_islet_rna.hg38.TPMs.unfiltered.MergedReplicates.txt')
tic()
tpm_final_merged = read.table(tablename)
toc()




tpm_final_merged = tpm_final_merged[rownames(res_rowdata),]
tpm_final_merged$gene_width_from_gencode41.gene_name = NULL
tpm_final_merged$gene_width_from_gencode41.row_name = NULL
tpm_final_merged$gene_width_from_gencode41.gene_width_from_gencode41 = NULL
tpm_final_merged$gene_id = NULL
tpm_final_merged$gene_gene_nameid = NULL
tpm_final_merged$gene_width_from_gencode41.gene_id = NULL
tpm_final_merged$gene_width_from_gencode41 = NULL
tpm_final_merged$gene_name = NULL
head(tpm_final_merged)

In [ ]:

tablename = paste0(indir, 'bulk_islet_rna.hg38.TPMs.unfiltered.unmerged.txt')
tic()
tpm_final_unmerged = read.table(tablename)
toc()




tpm_final_unmerged = tpm_final_merged[rownames(res_rowdata),]
tpm_final_unmerged$gene_width_from_gencode41.gene_name = NULL
tpm_final_unmerged$gene_width_from_gencode41.row_name = NULL
tpm_final_unmerged$gene_width_from_gencode41.gene_width_from_gencode41 = NULL
tpm_final_unmerged$gene_id = NULL
tpm_final_unmerged$gene_gene_nameid = NULL
tpm_final_unmerged$gene_width_from_gencode41.gene_id = NULL
tpm_final_unmerged$gene_name = NULL
head(tpm_final_unmerged)

In [ ]:
res_rowdata_tpm_unmerged = res_rowdata
res_rowdata_tpm_unmerged = cbind(res_rowdata, tpm_final_unmerged)
head(res_rowdata_tpm_unmerged)

In [ ]:
res_rowdata_tpm_merged = res_rowdata
res_rowdata_tpm_merged = cbind(res_rowdata, tpm_final_merged)
head(res_rowdata_tpm_merged)

In [ ]:

tic('merged save')
resoutfilename = paste0(outdir, 'bulk_islet_rna.hg38.Donor_plus_Treatment.Replicates_MERGED_by_sum.ANNOTATED.FILT_2Plus_Reads_per_Treat.with_TPMs.postDESeq.RESULTS.withRowdata_AND_TPM_merged.txt')
write.table(res_rowdata_tpm_merged, resoutfilename, quote=FALSE, sep='\t', col.names = TRUE, row.names=TRUE)
toc()


tic('merged save')
resoutfilename = paste0(outdir, 'bulk_islet_rna.hg38.Donor_plus_Treatment.Replicates_MERGED_by_sum.ANNOTATED.FILT_2Plus_Reads_per_Treat.with_TPMs.postDESeq.RESULTS.withRowdata_AND_TPM_merged.csv')
write.table(res_rowdata_tpm_merged, resoutfilename, quote=FALSE, sep=',', col.names = TRUE, row.names=TRUE)
toc()


In [ ]:

tic('unmerged save')
resoutfilename = paste0(outdir, 'bulk_islet_rna.hg38.Donor_plus_Treatment.Replicates_MERGED_by_sum.ANNOTATED.FILT_2Plus_Reads_per_Treat.with_TPMs.postDESeq.RESULTS.withRowdata_AND_TPM_unmerged.txt')
write.table(res_rowdata_tpm_unmerged, resoutfilename, quote=FALSE, sep='\t', col.names = TRUE, row.names=TRUE)
toc()


tic('unmerged save')
resoutfilename = paste0(outdir, 'bulk_islet_rna.hg38.Donor_plus_Treatment.Replicates_MERGED_by_sum.ANNOTATED.FILT_2Plus_Reads_per_Treat.with_TPMs.postDESeq.RESULTS.withRowdata_AND_TPM_unmerged.csv')
write.table(res_rowdata_tpm_unmerged, resoutfilename, quote=FALSE, sep=',', col.names = TRUE, row.names=TRUE)
toc()




In [ ]:
head(res_rowdata_tpm_unmerged)